In [ ]:
using PyCall
using PyPlot
using Statistics

yf = pyimport("yfinance")
np = pyimport("numpy")

include("OptimPortfolio.jl")
include("Utils.jl")

# Data donwload and basic analysis 

In [ ]:
start = "2013-01-01"
finish = "2017-01-01" 

start_test = "2017-01-01"
finish_test  ="2018-01-01"

assets = ["AAPL", "MSFT", "AMZN",  "TSLA", "GOOGL",  "GOOG", "UNH", "NVDA", "JNJ", "META"] #"^GSPC"

df = yf.download(assets, start, finish, progress=false)

df_test = yf.download(assets, start_test, finish_test, progress=false)

market = yf.download("^GSPC", start, finish, progress=false)

market_test = yf.download("^GSPC", start_test, finish_test, progress=false)

df_log_ret = (df["Adj Close"] / df["Adj Close"].shift(1)).apply(np.log)
df_test_log_ret = (df_test["Adj Close"] / df_test["Adj Close"].shift(1)).apply(np.log)

market_log_ret = (market["Adj Close"] / market["Adj Close"].shift(1)).apply(np.log)
market_test_log_ret = (market_test["Adj Close"] / market_test["Adj Close"].shift(1)).apply(np.log);

## Asset analysis

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
    CD
    EF""", figsize=(8*2, 6*3))

# Stock prices
df["Adj Close"].plot(ax=ax["A"])

ax["A"].legend(ncol=2)

df_test["Adj Close"].plot(ax=ax["B"])

ax["B"].legend(ncol=2)

#Returns
df["Adj Close"].pct_change().plot(ax=ax["C"])
#df_log_ret.plot(ax=ax["C"])

ax["C"].legend(ncol=2)

df_test["Adj Close"].pct_change().plot(ax=ax["D"])
#df_test_log_ret.plot(ax=ax["D"])

ax["D"].legend(ncol=2)

#Histogram of returns
for asset in assets
    df["Adj Close"][asset].pct_change().hist(ax=ax["E"], bins=30, alpha=0.8, density=true, label=asset)
    #df_log_ret[asset].hist(ax=ax["E"], bins=50, alpha=0.8, label=asset)
end

ax["E"].set_xlim(-0.2, 0.2)

ax["E"].legend(ncol=2)

for asset in assets
    df_test["Adj Close"][asset].pct_change().hist(ax=ax["F"], bins=20, alpha=0.8, density=true, label=asset)
    #df_test_log_ret[asset].hist(ax=ax["F"], bins=30, alpha=0.8, label=asset)
end

ax["F"].set_xlim(-0.2, 0.2)

ax["F"].legend(ncol=2)

## Market analysis 

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
    CD
    EF""", figsize=(8*2, 6*3))

# Stock prices
market["Adj Close"].plot(ax=ax["A"])

ax["A"].legend(ncol=2)

market_test["Adj Close"].plot(ax=ax["B"])

ax["B"].legend(ncol=2)

#Returns
market["Adj Close"].pct_change().plot(ax=ax["C"])
#market_log_ret.plot(ax=ax["C"])

ax["C"].legend(ncol=2)

market_test["Adj Close"].pct_change().plot(ax=ax["D"])
#market_test_log_ret.plot(ax=ax["D"])

ax["D"].legend(ncol=2)

#Histogram of returns
market["Adj Close"].pct_change().hist(ax=ax["E"], bins=50, alpha=0.8, label="S&P500")
#market_log_ret.hist(ax=ax["E"], bins=50, alpha=0.8, label="S&P500")

ax["E"].legend(ncol=2)

market_test["Adj Close"].pct_change().hist(ax=ax["F"], bins=30, alpha=0.8, label="S&P500")
#market_test_log_ret.hist(ax=ax["F"], bins=20, alpha=0.8, label="S&P500")

ax["F"].legend(ncol=2)

# Resample data to 1-month frequency

In [ ]:
freq = "1M"

df = df.resample(freq).mean()

df_test = df_test.resample(freq).mean()

market = market.resample(freq).mean()

market_test = market_test.resample(freq).mean()

df_log_ret = (df["Adj Close"] / df["Adj Close"].shift(1)).apply(np.log)
df_test_log_ret = (df_test["Adj Close"] / df_test["Adj Close"].shift(1)).apply(np.log)

market_log_ret = (market["Adj Close"] / market["Adj Close"].shift(1)).apply(np.log)
market_test_log_ret = (market_test["Adj Close"] / market_test["Adj Close"].shift(1)).apply(np.log);

# Initialize portfolios 

In [ ]:
returns = df["Adj Close"].pct_change().dropna().values
log_returns = df_log_ret.dropna().values

assets = [item for item in df["Adj Close"].columns]

portfolio = create_portfolio(returns, assets)
portfolio_log = create_portfolio(log_returns, assets);

# Mean-Variance portfolio optimization 

### Fixed returns 

In [ ]:
target_return = 0.03

return_p, risk_p, w_opt = MV_fixed_return(portfolio, target_return; method="DCP")

return_market = market["Adj Close"].pct_change().mean()
risk_market = sqrt(market["Adj Close"].pct_change().var())

println("Expected Returns Portfolio: ", return_p)
println("Risk Portfolio: ", risk_p)
#println("Weights:", w_opt)

println("\nExpected Return Market: ", return_market)
println("Risk Market: ", risk_market)

println("\nPortfolio-market return ratio: ", return_p/return_market)
println("Portfolio-market risk ratio: ", risk_p/risk_market)

### Fixed risk 

In [ ]:
target_risk = 0.035

return_p, risk_p, w_opt = MV_fixed_risk(portfolio, target_risk; method="DCP")

return_market = market["Adj Close"].pct_change().mean()
risk_market = sqrt(market["Adj Close"].pct_change().var())

println("Expected Returns Portfolio: ", return_p)
println("Risk Portfolio: ", risk_p)
#println("Weights:", w_opt)

println("\nExpected Return Market: ", return_market)
println("Risk Market: ", risk_market)

println("\nPortfolio-market return ratio: ", return_p/return_market)
println("Portfolio-market risk ratio: ", risk_p/risk_market)

# Efficient frontier 

In [ ]:
RetRisk, weights = MV_efficient_frontier(portfolio, 100);

RetRisk_log, weights_log = MV_efficient_frontier(portfolio_log, 100);

risks = sqrt.(diag(portfolio.Σ))
risks_log = sqrt.(diag(portfolio_log.Σ))

plt.figure(figsize=(8*2, 6*2))

plt.subplot(2, 2, 1)

for i in 1:length(portfolio.μ)

    plt.scatter(risks[i], portfolio.μ[i], s=100, label=assets[i])
    
end

plt.scatter(risk_market, return_market, s=100, color="k", marker="d", label="S&P500")

plt.plot(RetRisk[:, 2], RetRisk[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

plt.ylabel("Expected return", fontsize=20, labelpad=15)
plt.xlabel("Risk", fontsize=20, labelpad=15)

plt.legend(ncol=2)

plt.subplot(2, 2, 2)

for i in 1:length(portfolio.μ)

    plt.scatter(risks_log[i], portfolio_log.μ[i], s=100, label=assets[i])
    
end

plt.scatter(risk_market, return_market, s=100, color="k", marker="d", label="S&P500")

plt.plot(RetRisk_log[:, 2], RetRisk_log[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

plt.ylabel("Expected log return", fontsize=20, labelpad=15)
plt.xlabel("Risk", fontsize=20, labelpad=15)

plt.legend(ncol=2)

#
plt.subplot(2,2,3)

N = Int(10^6)

rets = zeros(N)
risks = zeros(N)
sharpe_ratio = zeros(N)

for i in 1 : N

    w = rand(length(portfolio.μ))

    w = w / sum(w)
   
    return_p = dot(w, portfolio.μ)
    risk_p = sqrt(transpose(w)*portfolio.Σ*w)
    
    rets[i] = return_p
    risks[i] = risk_p
    
    sharpe_ratio[i] = return_p / risk_p
    
end

plt.plot(RetRisk[:, 2], RetRisk[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

plt.scatter(risks, rets, c=sharpe_ratio, s=8)

plt.ylabel("Expected return", fontsize=20, labelpad=15)
plt.xlabel("Risk", fontsize=20, labelpad=15)

plt.colorbar()

#
plt.subplot(2,2,4)

N = Int(10^6)

rets = zeros(N)
risks = zeros(N)
sharpe_ratio = zeros(N)

for i in 1 : N

    w = rand(length(portfolio_log.μ))

    w = w / sum(w)
   
    return_p = dot(w, portfolio_log.μ)
    risk_p = sqrt(transpose(w)*portfolio_log.Σ*w)
    
    rets[i] = return_p
    risks[i] = risk_p
    
    sharpe_ratio[i] = return_p / risk_p
    
end

plt.plot(RetRisk_log[:, 2], RetRisk_log[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

plt.scatter(risks, rets, c=sharpe_ratio, s=8)

plt.ylabel("Expected log return", fontsize=20, labelpad=15)
plt.xlabel("Risk", fontsize=20, labelpad=15)

plt.colorbar()

plt.subplots_adjust(wspace=0.3, hspace=0.3)

# Portfolio testing 

Up to this moment we have optimized the weights of our portfolio for any desired level of returns and risk. However, this is done in a training (in-sample) set, but in the future things could change and our optimal weights could'nt be optimal any more. Here we check how expected returns and expected risk for the optimized weights could vary in the future by applying our optimal weights to a test (out of sample) set. We will show 2 results:

- An example of the out-of-sample performance of the in-sample optimized portfolio yielding a fixed return of 0.025

- The Yield Curve of all in-sample optimal portfolios (the ones on the efficient frontier) in the out-of-sample set.

In [ ]:
#EXAMPLE

#Test in "future" data
df_test_returns = df_test["Adj Close"].pct_change().dropna().values

target_return = 0.025

#Optimize in past data
ret_opt, risk_opt, w_opt = MV_fixed_return(portfolio, target_return)

returns_test = df_test_returns * w_opt
    
risk = round(sqrt(var(returns_test)), digits=3)

dates = [df_test["Adj Close"].index[i] for i in 2: length(df_test)]

mean_return = round(mean(returns_test), digits=4)
return_market = round(market_test["Adj Close"].pct_change().mean(), digits=4)
risk_market = round(sqrt(market_test["Adj Close"].pct_change().var()), digits=3);

#YIELD CURVE
portfolio_test = create_portfolio(df_test_returns, assets) #Portfolio(df_test_returns.mean().values, df_test_returns.cov().values)

target_returns = 0.021 : 0.001 : 0.048

final_risks = []
final_returns = []

expected_risks = []
expected_returns = []

for target_return in target_returns

    expected_return, expected_risk, w_opt = MV_fixed_return(portfolio, target_return)

    returns = df_test_returns * w_opt

    actual_risk = round(sqrt(var(returns)), digits=6)
    actual_mean_return = round(mean(returns), digits=6)
    
    append!(expected_risks, expected_risk)
    append!(expected_returns, expected_return)
    
    append!(final_risks, actual_risk)
    append!(final_returns, actual_mean_return)
    
end

RetRisk_test, weights = MV_efficient_frontier(portfolio_test, 50);

In [ ]:
fig, ax = plt.subplot_mosaic("AB", figsize=(8*2,6))

ax["A"].plot(dates, returns_test, lw=3, marker="o", ms=12, label="Portfolio simulated returns")

ax["A"].axhline(mean_return, color="k", lw=3, label="Portfolio: μ=$mean_return,  σ=$risk")
ax["A"].axhline(return_market, color="k", lw=3, ls="--", label="Market: μ=$return_market,  σ=$risk_market")

ax["A"].tick_params("x", rotation=45)

ax["A"].set_ylim(0, 0.07)

ax["A"].legend()

ax["B"].scatter(risk_market, return_market, color="k", s=100, marker="d", label="S&P500")

ax["B"].plot(expected_risks, expected_returns, marker="o", label="In-sample Efficient Frontier", color="k")
ax["B"].plot(final_risks, final_returns, marker="o", label="In-sample Optimized Portfolios")

ax["B"].plot(RetRisk_test[:, 2], RetRisk_test[:, 1], color="C1", lw=1, ls="-", marker="o", 
    label="Out of sample Efficient Frontier")

ax["B"].set_ylabel("Efficient return", fontsize=20, labelpad=15)
ax["B"].set_xlabel("Risk", fontsize=20, labelpad=15)

ax["B"].legend()

#=
#INSET
axins = ax["B"].inset_axes([0.62, 0.1, 0.35, 0.4])

axins.plot(final_risks[1:8], final_returns[1:8], marker="o", color="r")
axins.plot(final_risks[9:end-12], final_returns[9:end-12], marker="o", color="g")
axins.plot(final_risks[end-12:end-7], final_returns[end-12:end-7], marker="o", color="r")

axins.plot(RetRisk_test[end-12:end, 2], RetRisk_test[end-12:end, 1], color="C1", lw=1, ls="-", marker="o")

axins.set_xticks([])
axins.set_yticks([])

ax["B"].indicate_inset_zoom(axins, edgecolor="black")
=#

plt.subplots_adjust(wspace=0.3)

# Portfolio Evolutionary Optimization 

## Sanity check with MV model 

### Fixed return optimization 

In [ ]:
target_return = 0.03

return_p_DCP, risk_p_DCP, w_opt_DCP = MV_fixed_return(portfolio, target_return; method="DCP")
return_p_EO, risk_p_EO, w_opt_EO = MV_fixed_return(portfolio, target_return; method="EO")


println("Expected Returns Portfolio (DCP): ", return_p_DCP)
println("Expected Returns Portfolio (EO): ", return_p_EO)

println("\nRisk Portfolio (DCP): ", risk_p_DCP)
println("Risk Portfolio (EO): ", risk_p_EO)

### Fixed risk optimization 

In [ ]:
target_risk = 0.035

return_p_DCP, risk_p_DCP, w_opt_DCP = MV_fixed_risk(portfolio, target_risk; method="DCP")
return_p_EO, risk_p_EO, w_opt_EO = MV_fixed_risk(portfolio, target_risk; method="EO")

println("Expected Returns Portfolio (DCP): ", return_p_DCP)
println("Expected Returns Portfolio (EO): ", return_p_EO)

println("\nRisk Portfolio (DCP): ", risk_p_DCP)
println("Risk Portfolio (EO): ", risk_p_EO)

### Efficient Frontier 

In [ ]:
#Efficient frontier using Convex Optimization
@time RetRisk_DCP, weights_DCP = MV_efficient_frontier(portfolio, 50; method="DCP")

#Efficient frontier using Evolutionary Optimization
@time RetRisk_EO, weights_EO = MV_efficient_frontier(portfolio, 50; method="EO");

plt.plot(RetRisk_DCP[:, 2], RetRisk_DCP[:, 1], color="g", lw=3, zorder=1, label="DCP")
plt.scatter(RetRisk_EO[:, 2], RetRisk_EO[:, 1], color="r", label="EO")

plt.legend()

# Higher-Moment Portfolio Optimization 

Now the optimization problem is non-convex, so DCP can not be used. We will optimize the portfolios under MVSK model using Evolutionary Optimization.

### Example

In [ ]:
result_MV, w_opt_MV = MVSK(portfolio, [0.4, 0.6, 0.0, 0.0])

result_MVS, w_opt_MVS = MVSK(portfolio, [0.4, 0.4, 0.2, 0.0])

result_MVSK, w_opt_MVSK = MVSK(portfolio, [0.4, 0.4, 0.1, 0.1])

println("MV: ", result_MV)
println("MVS: ", result_MVS)
println("MVSK: ", result_MVSK)

## High-dimensional efficient frontier 

This optimization problem is highly computationally demanding, so we run the simulations in a cluster. Here we load the data and show some results.

In [ ]:
using DelimitedFiles

filenames = readdir("HD_efficient_frontier")

results = zeros((length(filenames), 4))
λs = zeros((length(filenames), 4))
weights = zeros((length(filenames), 10))

i = 1

for filename in filenames
   
    data, header = readdlm(string("HD_efficient_frontier/", filename), header=true)
   
    results[i, :] = data[1:4]
    λs[i, :] = data[5:8]
    weights[i, :] = data[9:end]
    
    i += 1
    
end

results_MV = results[(λs[:, 3] .== 0.0) .& (λs[:, 4] .== 0.0), :]
results_MS = results[(λs[:, 2] .== 0.0) .& (λs[:, 4] .== 0.0), :]
results_MK = results[(λs[:, 2] .== 0.0) .& (λs[:, 3] .== 0.0), :]

results_RS = results[(λs[:, 1] .== 0.0) .& (λs[:, 4] .== 0.0), :];

In [ ]:
plt.figure(figsize=(8,6))

plt.scatter(results_MV[:, 2], results_MV[:, 1], s=100, label="MV")

plt.scatter(results_MK[:, 2], results_MK[:, 1], s=100, label="MK")

plt.scatter(results_RS[:, 2], results_RS[:, 1], s=100, label="RS")

#plt.xlim(0.02, 0.1)

plt.legend()

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
CD""", figsize=(8*2,6*2))

ax["A"].scatter(results[:, 2], results[:, 1], c=results[:, 3], cmap="jet")

ax["B"].scatter(results[:, 2], results[:, 1], c=results[:, 4], cmap="jet")

ax["C"].scatter(results[:, 4], results[:, 3], c=results[:, 1], cmap="jet")
ax["D"].scatter(results[:, 4], results[:, 3], c=results[:, 2], cmap="jet")

# Higher moments 

In [ ]:
N = Int(10^4)

rets = zeros(N)
risks = zeros(N)
skews = zeros(N)
kurts = zeros(N)

sharpe_ratio = zeros(N)

@time for i in 1 : N

    w = rand(length(portfolio.μ))

    w = w / sum(w)
   
    return_p = dot(w, portfolio.μ)
    risk_p = sqrt(transpose(w)*portfolio.Σ*w)
    skew_p = skewness_portfolio(portfolio, w)
    kurt_p = kurtosis_portfolio(portfolio, w)
    
    rets[i] = return_p
    risks[i] = risk_p
    skews[i] = skew_p
    kurts[i] = kurt_p
    
    sharpe_ratio[i] = return_p / risk_p
    
end

RetRisk_DCP, weights_DCP = MV_efficient_frontier(portfolio, 50; method="DCP");

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
    CD""", figsize=(8*2, 6*2))

ax["A"].scatter(kurts, skews, c=rets, cmap="jet")
ax["A"].scatter(results[:, 4], results[:, 3], color="k", s=10)

ax["A"].set_xlim(0.1, 1.2)
ax["A"].set_ylim(-0.15, 0.3)

ax["B"].scatter(kurts, skews, c=risks, cmap="jet")
ax["B"].scatter(results[:, 4], results[:, 3], color="k", s=10)

ax["B"].set_xlim(0.1, 1.2)
ax["B"].set_ylim(-0.15, 0.3)

ax["C"].scatter(risks, rets, c=skews, cmap="jet")
ax["C"].plot(RetRisk_DCP[:, 2], RetRisk_DCP[:, 1], color="k", lw=3, alpha=0.2, label="Efficient frontier")

ax["D"].scatter(risks, rets, c=kurts, cmap="jet")
ax["D"].plot(RetRisk_DCP[:, 2], RetRisk_DCP[:, 1], color="k", lw=3, alpha=0.2, label="Efficient frontier")

# Auxiliar things 

In [ ]:
using DelimitedFiles

returns = df["Adj Close"].pct_change().dropna().values
log_returns = df_log_ret.dropna().values

assets = string.(zeros((1, 10)))

i = 0

for item in df["Adj Close"].columns
   
    i += 1
    
    assets[i] = item
    
end

f = open("returns.txt", "w")
f2 = open("log_returns.txt", "w")

writedlm(f, assets)
writedlm(f, returns)

writedlm(f2, assets)
writedlm(f2, log_returns)

close(f)
close(f2)

In [ ]:
λs = []

δ = 0.02

for i in 0.0 : δ : 1.0
    
    for j in 0.0 : δ : 1.0
        
        for k in 0.0 : δ : 1.0
            
            for l in 0.0 : δ : 1.0
                
                if i+j+k+l == 1.0
                   
                    append!(λs, [[i,j,k,l]])
                    
                end
                
            end
            
        end
        
    end
    
end

λs